In [1]:
## Load CSV data
# The data used in this tutorial are taken from the Titanic passenger list. 
# The model will predict the likelihood a passenger survived based on characteristics like age, gender, ticket class, and whether the person was traveling alone.
 
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [10]:
# Load data
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [11]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [12]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [13]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'male']
age                 : [ 0.75 27.   34.   30.   28.  ]
n_siblings_spouses  : [2 0 0 1 0]
parch               : [1 0 0 0 0]
fare                : [19.258 30.5    8.05  16.1    0.   ]
class               : [b'Third' b'First' b'Third' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'y']


In [14]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

sex                 : [b'male' b'female' b'female' b'male' b'male']
age                 : [28. 30.  9. 24. 28.]
n_siblings_spouses  : [1 0 4 2 0]
parch               : [0 0 2 0 0]
fare                : [15.85  93.5   31.275 24.15   7.229]
class               : [b'Third' b'First' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'B' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'n' b'y']


In [15]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)

age                 : [28. 15. 51. 26. 28.]
n_siblings_spouses  : [0 1 0 0 0]
class               : [b'Third' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [ ]:
# Data preprocessing 

In [16]:
# Continuous data

SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [30. 26.  1. 19. 18.]
n_siblings_spouses  : [0. 0. 5. 3. 2.]
parch               : [0. 0. 2. 2. 2.]
fare                : [  7.229   7.896  46.9   263.    262.375]


In [17]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [18]:
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [19]:
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print()
  print(labels.numpy())

[[ 19.      0.      2.     26.283]
 [ 38.      0.      0.     13.   ]
 [ 16.      0.      0.      7.75 ]
 [ 18.      1.      0.    108.9  ]
 [ 36.      0.      0.     13.   ]]

[1 0 1 0 0]


In [20]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'male' b'male' b'female']
age                 : [ 9. 36. 28. 48. 39.]
n_siblings_spouses  : [3 0 0 1 1]
parch               : [2 0 0 0 1]
fare                : [27.9   26.388  8.05  52.    79.65 ]
class               : [b'Third' b'First' b'Third' b'First' b'First']
deck                : [b'unknown' b'E' b'unknown' b'C' b'E']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'y' b'n' b'n']


In [21]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [23]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [24]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [25]:
show_batch(packed_train_data)

sex                 : [b'male' b'female' b'male' b'female' b'male']
class               : [b'Third' b'Second' b'Third' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'n' b'y']
numeric             : [[21.     0.     0.     8.663]
 [ 3.     1.     2.    41.579]
 [28.     0.     0.     7.896]
 [35.     1.     1.    20.25 ]
 [59.     0.     0.     7.25 ]]


In [ ]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [26]:
# Data Normalization

import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [27]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [37]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std


In [38]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000002BA47B7C400>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [40]:
example_batch['numeric']

KeyError: 'numeric'

In [41]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

ValueError: Feature numeric is not in features dictionary.

In [42]:
# Categorical data

CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}


In [46]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [47]:
# See what you just created.
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [48]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

ValueError: Feature alone is not in features dictionary.

In [49]:
# Combined preprocessing layer

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [50]:
print(preprocessing_layer(example_batch).numpy()[0])

ValueError: Feature alone is not in features dictionary.

In [51]:
# Build the model

model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1),
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [52]:
# Train, evaluate, and predict

train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [53]:
model.fit(train_data, epochs=20)

Instructions for updating:
Use `tf.data.Dataset.cardinality()
Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
126/126 [==============================] - 0s 2ms/step - loss: 0.5042 - accuracy: 0.7528
Epoch 2/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4113 - accuracy: 0.8246
Epoch 3/20
126/126 [==============================] - 0s 1ms/step - loss: 0.4019 - accuracy: 0.8293
Epoch 4/20
126/126 [==============================] - 0s 984us/step - loss: 0.3855 - accuracy: 0.8373
Epoch 5/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3797 - accuracy: 0.8341
Epoch 6/20
126/126 [==============================] - 0s 976us/step - loss: 0.3716 - accuracy: 0.8485
Epoch 7/20
126/126 [==============================] - 0s 992us/step - loss: 0.3606 - accuracy: 0.8453
Epoch 8/20
126/126 [==============================] - 0s 1ms/step - loss: 0.3580 - accuracy: 0.8485
Epoch 9/20
126/126 [==

In [54]:
test_loss, test_accuracy = model.evaluate(test_data)

print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

Consider rewriting this model with the Functional API.
53/53 [==============================] - 0s 1ms/step - loss: 0.4633 - accuracy: 0.8447


Test Loss 0.4632790982723236, Test Accuracy 0.8446969985961914


In [55]:
predictions = model.predict(test_data)

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))


Consider rewriting this model with the Functional API.
Predicted survival: 4.62%  | Actual outcome:  DIED
Predicted survival: 5.62%  | Actual outcome:  DIED
Predicted survival: 9.84%  | Actual outcome:  SURVIVED
Predicted survival: 9.74%  | Actual outcome:  DIED
Predicted survival: 6.38%  | Actual outcome:  SURVIVED
